# Al surface - Reduced cell


In this notebook we will do abinit calculation for a Aluminium surface.
For simplicity we will use convetional cell for the calculations.

Basic overview:
- Bulk - structure optimisation:
  - total energy
  - lattice parameters
- Surfaces:
  - the relaxation of surface atoms?
  - surface energy


In [ ]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline

import warnings 
import numpy as np
import matplotlib.pylab as plt
import plotly.graph_objects as go

from itertools import product

from abipy import abilab, flowtk

from pymatgen.core import Structure, Lattice
from pseudo_dojo import OfficialDojoTable

from jupyter_jsmol import JsmolView
from jupyter_jsmol.pymatgen import quick_view
from pymatgen_plotly import Figure

abilab.enable_notebook() # This line tells AbiPy we are running inside a notebook
warnings.filterwarnings("ignore")  # Ignore warnings

pseudos_table = OfficialDojoTable.from_dojodir("ONCVPSP-PBE-PDv0.4", accuracy='standard')


# Bulk

## Initial structure

In [ ]:
lattice_constant = 4.041265916093099 # angstrom
lattice_constant

In [ ]:
def fcc_conv(element:str, a:float):
    # lattice=Lattice.cubic(a),
    structure = Structure(
        lattice=Lattice([[a, 0, 0], [0, a, 0], [0, 0, a]]),
        species=4*[element],
        coords=[[0, 0, 0], [.5, .5, 0], [.5, 0, .5], [0, .5, .5]]
    )
    return structure

struct_conv = fcc_conv('Al', lattice_constant) 
struct_conv

## Surface energy of aluminum (100): changing the orientation of the unit cell¶

In order to study the Aluminum (100) surface, we will have to set up a supercell representing a slab. This supercell should be chosen as to be compatible with the primitive surface unit cell. The corresponding directions are `[-1 1 0]` and `[1 1 0]`. The direction perpendicular to the surface is `[0 0 1]`. There is no primitive cell of bulk aluminum based on these vectors, but a doubled cell. We will first compute the total energy associated with this doubled cell. This is not strictly needed, but it is a valuable intermediate step towards the study of the surface.

You might start from tbase4_3.in. You have to change rprim. Still, try to keep acell at the values of bulk aluminum that were determined previously. But it is not all: the most difficult part in the passage to this doubled cell is the definition of the k-point grid. Of course, one could just take a homogeneous simple cubic grid of k-points, but this will not correspond exactly to the k-point grid used in the primitive cell in tbase4_3.in. This would not be a big problem, but you would miss some error cancellation.

In [ ]:
def fcc_reduced(element: str, a: float):
    structure = Structure(
        lattice=Lattice([[a/2, -a/2, 0], [a/2, a/2, 0], [0, 0, a]]),
        species=2 * [element],
        coords=[[0, 0, 0], [.5, .5, .5]]
    )
    return structure

struct_initial = fcc_reduced('Al', lattice_constant)
struct_initial

In [ ]:
fig = Figure()
fig.add_structure(struct_conv)
fig.add_unitcell(struct_conv.lattice)
# fig.add_unitcell(struct_initial.lattice)
fig.show()

In [ ]:
# quick_view(struct_initial)

fig = Figure()
fig.add_structure(struct_initial, supercell=(1,1,1))
fig.add_unitcell(struct_initial.lattice)
fig.show()

## Convergence (k-point, tsmear)

## The convergence study with respect to k points and broadening

Note that there is usually a STRONG cross-convergence effect between the number of k-points (sampling of the Brillouin zone) and the value of the broadening.
The right procedure is: for each value of broadening, to get the convergence with respect to the number of k points `nkpt`, then to compare the k-point converged values for different values of `tsmear`.

In what follows, we will restrict ourselves to the grids with `nkpt=2` (`ngkpt 2 2 2`), `10`  (`ngkpt 4 4 4`) and `28`  (`ngkpt 6 6 6`).

Now we use `relax_input` to generate multiple inputs with different values of `tsmear` and `nksmall`
and we pass the input objects to the Flow constructor.
To keep things as simple as possible, we use independent tasks ...

In [ ]:
# tsmear_list = [0.01, 0.02, 0.03, 0.04]
# nksmall_list = [2, 4, 6]

# inputs     = [relax_input(struct_prim, pseudo_table, tsmear, nksmall) for tsmear, nksmall in product(tsmear_list, nksmall_list)]


In [ ]:
# # Build flow form inputs.
# # Note the Flow.from_inputs is a simplified interface that, by default, builds tasks
# # for Ground-state calculation (GsTask).
# # Here we are performing a structural relaxation so we have to specify the task class explicitly.
# # AbiPy will use this piece of information to handle the restart of the RelaxTask that differs
# # from the one provided by GsTask.
    
# flow = flowtk.Flow.from_inputs('flow_al_relax_tsmear_nkpt', inputs=inputs, task_class=flowtk.RelaxTask)
# # flow.show_status()
# # flow.get_graphviz()

In [ ]:
# flow.rmtree()
# scheduler = flow.make_scheduler()
# scheduler.start()

In [ ]:
# robot = abilab.GsrRobot.from_dir("flow_al_relax_tsmear_nkpt")
# data = robot.get_dataframe()
# print(data.keys())

In [ ]:
# x_labels, x_inds = np.unique(data['nkpt'], return_inverse=True)
# y_labels, y_inds = np.unique(data['tsmear'], return_inverse=True)

# z = np.zeros((len(x_labels), len(y_labels)))
# for x_ind, y_ind, d in zip(x_inds, y_inds, data['energy']):
#     z[x_ind, y_ind] = d
    
# fig = go.Figure(data=[go.Surface(x=x_labels, y=y_labels, z=z)])
# fig.show()

## Single SCF calculation

In [ ]:
def build_inp_scf(structure, ecut=20, tsmear=0.01, shifted=False):
    """Generate input for optimization of the lattice parameter
    at fixed number of k points and broadening.
    """

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)
    

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    # ecut_hint = max(ps.dojo_report["hints"]["normal"]["ecut"] for ps in inp.pseudos)
    # ecut_hint = max(ps.hint_for_accuracy("normal").ecut for ps in inp.pseudos)
    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid (equivalent to a 16*16*16 grid on a convetnional cell)
    inp["kptopt"] = 1
    inp["ngkpt"] = [8, 8, 8] 

    inp["nshiftk"] = 2
    if shifted:
        inp["shiftk"] = [[.5, .0, .5], [0., .5, .5]]            
    else:
        inp["shiftk"] = [[0., 0., 0.], [.5, .5, 0.]]

    inp["occopt"] = 6
    # inp["nband"] = 3 * len(structure) + 6 # NOTE: This value is specific for Al

    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_scf = build_inp_scf(struct_initial)
# inp_scf

In [ ]:
def build_flow_scf(inp, workdir, force=False):

    flow = flowtk.Flow.from_inputs(workdir, inputs=inp)

    if force:
        flow.rmtree()

    return flow

flow_scf = build_flow_scf(inp_scf, "reduced_cell/00_scf", force=True)

try:
    scheduler = flow_scf.make_scheduler()
    scheduler.start()
except:
    pass


In [ ]:
out_scf = abilab.abiopen('reduced_cell/00_scf/w0/t0/outdata/out_GSR.nc')
out_scf.energy

## Relaxation

In [ ]:
def build_inp_relax(structure, ecut=20, tsmear=0.01, shifted=False):
    """Generate input for optimization of the lattice parameter
    at fixed number of k points and broadening.
    """

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid (equivalent to a 16*16*16 grid on a convetnional cell)
    inp["kptopt"] = 1
    inp["ngkpt"] = [8, 8, 8] 

    inp["nshiftk"] = 2
    if shifted:
        inp["shiftk"] = [[.5, .0, .5], [0., .5, .5]]            
    else:
        inp["shiftk"] = [[0., 0., 0.], [.5, .5, 0.]]

    inp["occopt"] = 6
    inp["nband"] = 3 * len(structure) + 3 # NOTE: This value is specific for Al

    ## Optimization of the lattice parameters
    inp["optcell"] = 1
    inp["ionmov"] = 2

    inp["ntime"] = 10
    inp["dilatmx"] = 1.05
    inp["ecutsm"] = 0.5

    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_relax=build_inp_relax(struct_initial)
# inp_relax

In [ ]:
def build_flow_relax(inp, workdir, force=False):

    flow = flowtk.Flow.from_inputs(workdir, inputs=inp)

    if force:
        flow.rmtree()

    return flow

flow_relax = build_flow_relax(inp_relax, "reduced_cell/01_relax", force=False)

try:
    scheduler = flow_relax.make_scheduler()
    scheduler.start()
except:
    pass


In [ ]:
abo = abilab.abiopen("reduced_cell/01_relax/w0/t0/run.abo")

struct_relaxed = abo.final_structure
struct_relaxed

In [ ]:
lattice_constant_relaxed = struct_relaxed.lattice.c
lattice_constant_relaxed

As you can see, we essentialy have a standard input to perform a GS calculation. This object will represent
the **building block** for our DFPT calculation with AbiPy.

If this is not your first time you use the DFPT part of Abinit, you already know that phonon calculations
require an initial GS run to produce the `WFK` file 
followed by a DFPT run that reads the `WFK` file and solves the Sternheimer equations for $N_{\text{irred}}(q)$ 
atomic perturbations where $N_{\text{irred}}$ is the number of independent atomic displacements (assuming $q$ belongs to the k-mesh).

If you try to do a convergence study wrt `ecut` **without multi-datasets**, you will likely start from an initial GS input file with a given value of `ecut`, use it as a template to generate the DFPT input files, create symbolic 
links to the `WFK` file produced in the first GS step and then instruct Abinit to read this file with `irdwfk`.
Once you have a set of input files that work for a particular `ecut`, one can simply replicate the set of 
directories and files and use a script to change the value of `ecut` in the input files.
Then, of course, one has to run the calculations manually, collect the results and produce nice plots to understand
what is happening.

This approach is obviously boring and error-prone if you are a human being but it is easy to implement in an algorithm 
and machines do not complain if they have a lot of repetive work to do!
There are also several **technical advantages** in using this **task-based approach vs multi-datasets** but we discuss this point in more details afterwards. 

If the machine could speak, it will tell you: give me an object that represents an input for GS calculations,
give me the list of q-points you want to compute as well as the parameters that must be changed in the initial input 
and I will generate a `Flow` for DFPT calculations.
This logic appears so frequenty that we decided to encapsulate it in the `flowtk.phonon_conv_flow` factory function: 

In [ ]:
# def build_flow_alas_ecut_conv(options):
#     """
#     Build and return Flow for convergence study of phonon frequencies at Gamma as function of ecut.
#     """
#     scf_input = make_scf_input()
#     return flowtk.phonon_conv_flow("flow_alas_ecut_conv", scf_input, qpoints=(0, 0, 0), params=["ecut", [4, 6, 8]])



This convergence study at $\Gamma$ thus reveals that our pseudos require 
an `ecut` >= 6 Ha to get reasonably converged phonon frequencies at $\Gamma$.
In what follows, we assume that also the modes at the other $q$-points present a similar
convergence behaviour and we use `ecut` = 6 Ha to keep the computational cost low. 

In [ ]:
def build_inp_dftp(structure, ecut=20, tsmear=0.01, shifted=False):

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid
    inp["kptopt"] = 1
    inp["ngkpt"] = [8, 8, 8] 

    inp["nshiftk"] = 2
    if shifted:
        inp["shiftk"] = [[.5, .0, .5], [0., .5, .5]]            
    else:
        inp["shiftk"] = [[0., 0., 0.], [.5, .5, 0.]]

    inp["occopt"] = 6
    inp["nband"] = 3 * len(structure) + 3 # NOTE: This value is specific for Al

    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_dftp = build_inp_dftp(struct_relaxed)

In [ ]:
def build_flow_dftp(inp, workdir, force=False):

    """Build and return a Flow to compute the dynamical matrix on a (2, 2, 2) qmesh
    as well as DDK and Born effective charges.
    The final DDB with all perturbations will be merged automatically and placed
    in the Flow `outdir` directory.
    """

    flow = flowtk.PhononFlow.from_scf_input(
        workdir, inp, ph_ngqpt=(4, 4, 4), with_becs=False
    )

    if force:
        flow.rmtree()

    return flow

flow_dftp = build_flow_dftp(inp_dftp, "reduced_cell/02_dftp")

try:
    scheduler = flow_dftp.make_scheduler()
    scheduler.start()
except:
    pass


In [ ]:
ddb = abilab.abiopen("reduced_cell/02_dftp/outdata/out_DDB")
print(ddb)

In [ ]:
ddb.anaget_ifc(asr=0, workdir='reduced_cell/02_dfpt_ifc')

In [ ]:
?ddb.anaget_ifc

In [ ]:
phbst_file, phdos_file = ddb.anaget_phbst_and_phdos_files(asr=0, ndivsm=10, nqsmall=16, lo_to_splitting=False, workdir='reduced_cell/02_dftp_phbst')

# Extract the phonon bands and the phonon DOS from phbst_file and phdos_file
phbands = phbst_file.phbands 
phdos = phdos_file.phdos

In [ ]:
phbands = abilab.abiopen('reduced_cell/02_dftp_phbst/run.abo_PHBST.nc').phbands
phbands

In [ ]:
phbands.plot()

In [ ]:
?ddb.anaget_phbst_and_phdos_files

# Surfaces

## Surface 100

In [ ]:
def fcc100_reduced(element: str, a: float, n: int, vacuum: float, centered=True):

    h = (n - 1) * a / 2 + vacuum

    coords = np.zeros((n, 3))
    coords[1::2, 0] = a/2
    coords[:, 2] = [i*a/2 for i in range(n)]
    # coords[:, 2] = np.arange(n) * a / 2

    if centered:
        coords[:, 2] += vacuum/2

    structure = Structure(
        lattice=Lattice([[a/2, -a/2, 0], [a/2, a/2, 0], [0, 0, h]]),
        species=n * [element],
        coords=coords,
        coords_are_cartesian=True,
    )
    return structure


In [ ]:
struct_fcc100 = fcc100_reduced('Al', lattice_constant_relaxed, n=2, vacuum=lattice_constant_relaxed/2, centered=False) 
struct_fcc100

In [ ]:
struct_relaxed

In [ ]:
struct_fcc100 = fcc100_reduced('Al', lattice_constant_relaxed, n=5, vacuum=5*lattice_constant_relaxed/2, centered=True) 
struct_fcc100

In [ ]:
18/4

In [ ]:
# quick_view(struct_fcc100, supercell=(3,3,2))

fig = Figure()
fig.add_structure(struct_fcc100, supercell=(3,3,2))
fig.add_unitcell(struct_fcc100.lattice)
fig.show()



In [ ]:
def build_inp_fcc100_scf(structure, ecut=20, tsmear=0.01, shifted=False):
    """Generate input for optimization of the lattice parameter
    at fixed number of k points and broadening.
    """

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid (equivalent to a 16*16*16 grid on a convetnional cell)
    inp["kptopt"] = 1
    inp["ngkpt"] = [8, 8, 1] 

    inp["nshiftk"] = 2
    if shifted:
        inp["shiftk"] = [[.5, .0, .5], [0., .5, .5]]   # TODO: To check the last shift value!?!?   
    else:
        inp["shiftk"] = [[0., 0., 0.], [.5, .5, 0.]]

    inp["occopt"] = 6
    inp["nband"] = 3 * len(structure) + 3 # NOTE: This value is specific for Al

    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_fcc100_scf = build_inp_scf(struct_fcc100)
# inp_fcc100_scf

In [ ]:
flow_scf = build_flow_scf(inp_fcc100_scf, "reduced_cell/03_fcc100_scf", force=False)

try:
    scheduler = flow_scf.make_scheduler()
    scheduler.start()
except:
    pass


## Convergence (number of layers, surface energy, heigth of vacuum)

### Calculating the surface formation energy of a crystalline solid

In density functional theory, surface energy can be calculated from the following expression:

$$\gamma = \frac{E_\text{slab} - N E_\text{bulk}}{2A}$$

where
- $E_\text{slab}$ is the total energy of surface slab obtained using density functional theory.
- $N$ is the number of atoms in the surface slab. 
- $E_\text{bulk}$ is the bulk energy per atom.
- $A$ is the surface area.

For a slab, we have two surfaces and they are of the same type, which is reflected by the number 2 in the denominator. To guarantee this, we need to create the slab carefully to make sure that the upper and lower surfaces are of the same type.

## Convergence study at $\Gamma$




In [ ]:
?flowtk.phonon_conv_flow

Each group represents a `Workflow` and consists of one `ScfTask`(red circle) that solves the `KS` equations self-consistently producing a `WFK` file that will be used by the two children (`PhononTasks` - blue circles)
to compute the first-order change of the wavefunctions due to one of the *irreducible* atomic pertubations.

Note that `phonon_conv_flow` invokes Abinit under the hood to get the list of irreducible perturbations 
and uses this information to build the flow.
This explains why we have two `PhononTasks` per $q$-point instead of the total number of phonon modes that 
equals $3*N_{atom}=6$.

Perhaps a table with the values of the input variables associated to the DFPT perturbation will help.
`None` means that the variable is not defined in that particular input.

In [ ]:
# def build_flow_alas_ecut_conv(options):
#     """
#     Build and return Flow for convergence study of phonon frequencies at Gamma.
#     TODO:  wokaround: params=["ecut", [20]]
#     """
#     scf_input = make_scf_input()
#     return flowtk.phonon_conv_flow("flow_al_ecut_conv", scf_input, qpoints=(0, 0, 0),  params=["ecut", [16, 20, 26]])


# flow = build_flow_alas_ecut_conv(options=None)
# flow.show_status()

# flow.show_info()


There are several output files located inside the `outdata` directories:

In [ ]:
robot = abilab.DdbRobot.from_dir_glob("./flow_alas_ecut_conv/w*/outdata/")
robot

## Relaxation (with fixed inner layers)

In [ ]:
# def relax_100_surface_input(n, vacuum, n_frozen=None):
#     """Generate input for optimization of the lattice parameter
#     at fixed number of k points and broadening. 
#     """
#     structure = fcc100('Al', relaxed_lattice_constant, n=n, vacuum=vacuum) 
#     inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

#     ## Definition of the k-point grid
#     # Automatically select ngkpt, shift, kptopt for the sampling of the BZ
#     # from the lattice and the number of divisions
#     inp['ngkpt'] = [4, 4, 1]
#     inp['nshiftk'] = 2
#     inp['shiftk'] = [[.5, 0, 0], [0, .5, 0]]
    
#     ## Definition of the unit cell
#     # This input variable allows to use non-primitive unit
#     # cells. Please, do not use it in other cases,
#     # you might miss a primitive cell, faster to handle.
#     inp['chkprim'] = 0
#     inp['chksymbreak'] = 0 # ??? WARNING: Found potentially symmetry-breaking value of tnons ...
    
#     ## Definition of the planewave basis set
#     # Maximal kinetic energy cut-off, in Hartree (16, 20, 26)
#     inp['ecut'] = 20 

#     ## Exchange-correlation functional
#     # LDA Teter Pade parametrization
#     # inp['ixc'] = 1  # ??? WARNING: Pseudopotential file pspxc: 11, not equal to input ixc: 1.
#     # PBE Perdew-Burke-Ernzerhof GGA functional 
#     inp['ixc'] = 11
    
#     ## Definition of occupation numbers
#     inp['occopt'] = 4
#     inp['tsmear'] = 0.04
# #     inp['nband'] = 2*(n + 2)   # ??? WARNING

#     ## Optimization of the lattice parameters
#     inp['optcell'] = 0 
#     inp['ionmov'] = 2
#     inp['ntime'] = 10
#     inp['dilatmx'] = 1.05
#     inp['ecutsm'] = 0.5

        
#     # Fixing the position of cntral atoms 
#     if n_frozen:
#         assert n%2 == n_frozen%2 and n > n_frozen
#         inp['natfix'] = n_frozen
#         inp['iatfix'] = list(range((n - n_frozen)//2 + 1, (n + n_frozen)//2 + 1))
        
    
#     ## Definition of the SCF procedure
#     # Maximal number of SCF cycles
#     inp['nstep'] = 50 

#     # Will stop when, twice in a row, the difference
#     # between two consecutive evaluations of total energy
#     # differ by less than toldfe (in Hartree)
#     # This value is way too large for most realistic studies of materials
# #     inp['toldfe'] = 1e-6 # low
#     # inp['tolvrs'] = 1e-8 # low
#     # inp['toldfe'] = 1e-8 # probably ok
#     inp['tolvrs'] = 1e-10 # probably ok
    
    
#     return inp


## DFTP

In [ ]:
def build_inp_fcc100_dftp(structure, ecut=20, tsmear=0.01, shifted=False):

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid
    inp["kptopt"] = 1
    inp["ngkpt"] = [8, 8, 1] 

    inp["nshiftk"] = 2
    if shifted:
        inp["shiftk"] = [[.5, .0, .5], [0., .5, .5]]   # NOTE: cheking last shift!?!?   
    else:
        inp["shiftk"] = [[0., 0., 0.], [.5, .5, 0.]]

    inp["occopt"] = 6
    inp["nband"] = 3 * len(structure) + 3 # NOTE: This value is specific for Al

    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_fcc100_dftp = build_inp_fcc100_dftp(struct_fcc100)

## Phonon band structure of Al
[[back to top](#top)]

Now we are finally ready for the calculation of the vibrational spectrum of $AlAs$.
We already managed to run DFPT calculations at $\Gamma$ with different values of `ecut` and the
steps required to get a full band structure are not that different, provided that 
the following differences are taken into account:

- we need the dynamical matrix $D(q)$ on a homogeneous mesh so that it is possible to calculate $D(R)$
  in anaddb via Fourier transform and then phonon frequencies for arbitrary q-points via Fourier interpolation
  
- $AlAs$ is a polar semiconductor so we need to include the LO-TO splitting for $q \rightarrow 0$ that, in turns,
  requires the DFPT computation of the Born effective charges and of the dielectric constant.


In AbiPy, these concepts are translated in an easy-to-use API in which you pass an initial `AbinitInput` object,
you specify the q-mesh for phonons in terms of `ph_nqpt` and activate the computation of the 
Born effective charges with the boolean flag `with_becs`.

Let's have a look at the code (as usual there are more comments than lines of code):

Note that there are a lot of things happening under the hood here.

First of all, AbiPy generates `PhononTasks` only for the $q$-points in the 
irreducible wedge of the Brillouin zone corresponding to `ph_ngqpt`.
Moreover, for a given $q$-point, only the irreducible atomic perturbations are explicitly computed
since the other atomic perturbations can be reconstructed by symmetry.
Fortunately you do not have to care about all these technical details as AbiPy and Abinit 
will automate the whole procedure.

Remember that the $q$-point mesh cannot be chosen arbitrarily
since all $q$ wavevectors should connect two $k$ points of the grid used for the electrons.

It is also worth stressing that the computational cost of the DFPT run depends on the q-point 
since only those symmetries that preserve the q-point as well as the direction of the perturbation 
can be employed (calculations at $\Gamma$ are therefore much faster than other q-points).

In [ ]:
def build_flow_dftp(inp, workdir, force=False):

    """Build and return a Flow to compute the dynamical matrix on a (2, 2, 2) qmesh
    as well as DDK and Born effective charges.
    The final DDB with all perturbations will be merged automatically and placed
    in the Flow `outdir` directory.
    """

    flow = flowtk.PhononFlow.from_scf_input(
        workdir, inp, ph_ngqpt=(4, 4, 1), with_becs=False
    )

    if force:
        flow.rmtree()

    return flow

flow_dftp = build_flow_dftp(inp_fcc100_dftp, "reduced_cell/04_fcc100_dftp")

try:
    scheduler = flow_dftp.make_scheduler()
    scheduler.start()
except:
    pass


## Post-processing the results

Our flow is completed and we have the final DDB file with all the $q$-points and all the independent atomic perturbations. 
Let's open this DDB file with:

In [ ]:
ddb = abilab.abiopen("reduced_cell/04_fcc100_dftp/outdata/out_DDB")
print(ddb)

The `DdbFile` object provides an easy-to-use interface that invokes `anaddb` to post-process
the data stored in the DDB file.

`anacompare_phdos`, for example, computes the phonon DOS with different $q$-meshes.
Each mesh is defined by a single integer, `nqsmall`, that gives the number of 
divisions used to sample the smallest reciprocal lattice vector. 
The number of divisions along the other directions are chosen so that proportions are preserved:

In [ ]:
c = ddb.anacompare_phdos(nqsmalls=[4, 8, 12, 16])
c.plotter.combiplot();

To function `anaget_phbst_and_phdos_files` allows one to compute the phonon band structure on an automatically defined $q$-path as well as the the phonon DOS:

In [ ]:
?ddb.anaget_phbst_and_phdos_files

In [ ]:
phbst_file, phdos_file = ddb.anaget_phbst_and_phdos_files(asr=0, ndivsm=10, nqsmall=16, lo_to_splitting=False, workdir='reduced_cell/04_fcc100_dftp_phbst')



In [ ]:
phdos_file = abilab.abiopen('reduced_cell/04_fcc100_dftp_phbst/run.abo_PHDOS.nc')
phbst_file = abilab.abiopen('reduced_cell/04_fcc100_dftp_phbst/run.abo_PHBST.nc')


In [ ]:
# Extract the phonon bands and the phonon DOS from phbst_file and phdos_file
phbands = phbst_file.phbands 
phdos = phdos_file.phdos

In [ ]:
phbands.plot()

In [ ]:
phbands.plot();

In [ ]:
phbands.qpoints.plot();

In [ ]:
phbands.plot_with_phdos(phdos);

In [ ]:
phbands.num_branches

In [ ]:
iqpt, imode = 0, 4
phbands.phdispl_cart[iqpt, imode,:].reshape(-1,3)

In [ ]:
phbands.plot_phdispl_cartdirs(0)

In [ ]:
phbands.qpoints.frac_coords()

In [ ]:
phbands.create_xyz_vib(0, 'test_vib.xyz', max_supercell=4)

In [ ]:
?phbands.structure.get_smallest_supercell

In [ ]:

max_supercell = [16,16,16]
for qpoint in phbands.qpoints.frac_coords:
    try:
        print(phbands.structure.get_smallest_supercell(qpoint, max_supercell))
    except:
        print(qpoint)

In [ ]:
from abipy.core.abinit_units import phfactor_ev2units

try:
    phfreqs = ddb.anaget_phmodes_at_qpoint(qpoint=[0,0,0], asr=0, workdir='reduced_cell/04_fcc100_dftp_phonon').phfreqs
except:
    phbands = abilab.abiopen('reduced_cell/04_fcc100_dftp_phonon/run.abo_PHBST.nc').phbands

phbands.phfreqs * phfactor_ev2units("cm-1")

In [ ]:
phbands.dyn_mat_eigenvect.shape

In [ ]:
# phbst = abilab.abiopen('reduced_cell/04_fcc100_dftp_phonon/run.abo_PHBST.nc')


In [ ]:
ifc = ddb.anaget_ifc(workdir='reduced_cell/04_fcc100_dftp_ifc')


In [ ]:
ifc.get_ifc_cartesian()


In [ ]:
d

In [ ]:
d.shape

In [ ]:
f.shape

In [ ]:
?ifc.get_ifc_cartesian

In [ ]:
ifc

In [ ]:
?abilab.abiopen

In [ ]:
ifc._structure

In [ ]:
fig = Figure()
fig.add_structure(ifc._structure, supercell=[2,2,1])
fig.add_unitcell(ifc._structure.lattice)

fig.show()

In [ ]:
ifc.neighbours_indices

In [ ]:
ifc.ifc_cart_coord_short_range

In [ ]:
ifc.local_vectors[0][1]

In [ ]:
np.unique(ifc.neighbours_indices[1])

In [ ]:
ifc.get_ifc_local()

In [ ]:
ifc.get_ifc_cartesian(atom_indices=[0])[]

In [ ]:
ifc.plot_longitudinal_ifc(atom_indices=[3])

In [ ]:
ifc.plot_longitudinal_ifc_short_range()

In [ ]:
ifc.get_ifc_cartesian()


In [ ]:
ifc

## Supercell calculation

In [ ]:
2x2x1

In [ ]:
def build_inp_fcc100_dftp_supercell(structure, supercell=None, ecut=20, tsmear=0.01, shifted=False):

    if supercell is not None:
        structure = structure.copy()
        structure.make_supercell(supercell)

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid
    inp["kptopt"] = 1
    
    if supercell is not None:
        inp["ngkpt"] = [8//supercell[0], 8//supercell[1], 1//supercell[2]] 
    else:
        inp["ngkpt"] = [8, 8, 1] 

    inp["nshiftk"] = 2
    if shifted:
        inp["shiftk"] = [[.5, .0, .5], [0., .5, .5]]   # NOTE: cheking last shift!?!?   
    else:
        inp["shiftk"] = [[0., 0., 0.], [.5, .5, 0.]]

    inp["occopt"] = 6
    inp["nband"] = 3 * len(structure) + 3 # NOTE: This value is specific for Al

    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_fcc100_dftp_supercell = build_inp_fcc100_dftp_supercell(struct_fcc100, supercell=[2,2,1])
# inp_fcc100_dftp_supercell

In [ ]:
def build_flow_dftp(inp, workdir, force=False):

    """Build and return a Flow to compute the dynamical matrix on a (2, 2, 2) qmesh
    as well as DDK and Born effective charges.
    The final DDB with all perturbations will be merged automatically and placed
    in the Flow `outdir` directory.
    """

    flow = flowtk.PhononFlow.from_scf_input(
        workdir, inp, ph_ngqpt=(2, 2, 1), with_becs=False
    )

    if force:
        flow.rmtree()

    return flow

# flow_dftp = build_flow_dftp(inp_fcc100_dftp_supercell, "reduced_cell/05_fcc100_221_dftp", force=False)

# try:
#     scheduler = flow_dftp.make_scheduler()
#     scheduler.start()
# except:
#     pass
# flow_dftp.allocate()
# flow_dftp.build_and_pickle_dump()

In [ ]:
inp_fcc100_441_dftp = build_inp_fcc100_dftp_supercell(struct_fcc100, supercell=[4,4,1])

def build_flow_dftp(inp, workdir, force=False):

    """Build and return a Flow to compute the dynamical matrix on a (2, 2, 2) qmesh
    as well as DDK and Born effective charges.
    The final DDB with all perturbations will be merged automatically and placed
    in the Flow `outdir` directory.
    """

    flow = flowtk.PhononFlow.from_scf_input(
        workdir, inp, ph_ngqpt=(1, 1, 1), with_becs=False
    )

    if force:
        flow.rmtree()

    return flow

flow_dftp = build_flow_dftp(inp_fcc100_441_dftp, "reduced_cell/05_fcc100_441_dftp", force=False)
flow_dftp.allocate()
flow_dftp.build_and_pickle_dump()

In [ ]:
ddb = abilab.abiopen('reduced_cell/05_fcc100_221_dftp/outdata/out_DDB')

In [ ]:
phbst_file, phdos_file = ddb.anaget_phbst_and_phdos_files(asr=0, ndivsm=10, nqsmall=8, lo_to_splitting=False, workdir='reduced_cell/05_fcc100_221_dftp_phbst')



In [ ]:
from abipy.core.abinit_units import phfactor_ev2units

try:
    phfreqs = ddb.anaget_phmodes_at_qpoint(qpoint=[0,0,0], asr=0, workdir='reduced_cell/05_fcc100_221_dftp_phonon').phfreqs
except:
    phbands = abilab.abiopen('reduced_cell/05_fcc100_221_dftp_phonon/run.abo_PHBST.nc').phbands

phbands.phfreqs * phfactor_ev2units("cm-1")

In [ ]:
# def surface_area(structure):
#     """Calculates the surface area of the slab
#     """
#     m = structure.lattice.matrix
#     return np.linalg.norm(np.cross(m[0], m[1]))

# def surface_energy(structure, energy_slab, energy_bulk):
#     """Calculates the surface energy of the slab
#     """
#     A = surface_area(structure)
#     N = len(structure)
#     return (energy_slab - N * energy_bulk) / (2 * A)

In [ ]:
# surface_area(abo.structure)

In [ ]:
# surface_energy(abo.structure, abo.energy, total_energy)

### Optimise the height of vacuum

In [ ]:

# vacuum_list = np.linspace(relaxed_lattice_constant / 2, 5 * relaxed_lattice_constant / 2, 7)
# vacuum_list

# # Build flow form inputs.

# inputs = [relax_100_surface_input(n=6, vacuum=vacuum) for vacuum in vacuum_list]

# Build flow form inputs.
# Note the Flow.from_inputs is a simplified interface that, by default, builds tasks
# for Ground-state calculation (GsTask).
# Here we are performing a structural relaxation so we have to specify the task class explicitly.
# AbiPy will use this piece of information to handle the restart of the RelaxTask that differs
# from the one provided by GsTask.
    
# flow = flowtk.Flow.from_inputs('05_relax_100_6_vacuum', inputs=inputs, task_class=flowtk.RelaxTask)
# flow.show_status()
# # flow.get_graphviz()

In [ ]:
# robot = abilab.GsrRobot.from_dir("05_relax_100_6_vacuum")

# energies = []
# surface_eneries = []

# for ind, calc in sorted(robot.items()):
#     energies.append(calc.energy)
#     surface_eneries.append(surface_energy(calc.structure, calc.energy, total_energy))

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(vacuum_list, energies, 'o-')
# plt.title('Converence of the total energy by separating the surfaces')
# plt.xlabel('Height of the vacuum  [A]')
# plt.ylabel('Energy per atom [eV]')

# plt.show()

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(vacuum_list, surface_eneries, 'o-')
# plt.title('Converence of the surface energy by separating the surfaces')
# plt.xlabel('Height of the vacuum [A]')
# plt.ylabel('Surface energy [eV]')

# plt.show()

Conclusion: A vacuum with the $7$ Angstom is good enough.

### Different number of layers

In [ ]:
# # Build flow form inputs.

# n_list = range(1, 10)
# inputs = [relax_100_surface_input(n=n, vacuum=7.0) for n in n_list]

# flow = flowtk.Flow.from_inputs('06_relax_100_nlayers', inputs=inputs, task_class=flowtk.RelaxTask)
# # flow.show_status()
# # flow.get_graphviz()

In [ ]:
# robot = abilab.GsrRobot.from_dir("06_relax_100_nlayers/w0")

# energies = []
# surface_eneries = []

# for ind, calc in sorted(robot.items()):
#     energies.append(calc.energy_per_atom)
#     surface_eneries.append(surface_energy(calc.structure, calc.energy, total_energy))

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_list, energies, 'o-')
# plt.title('Convergence of total energy')
# plt.xlabel('# of layers')
# plt.ylabel('Energy per atom [eV]')

# plt.show()

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_list, surface_eneries, 'o-')
# plt.title('Convergence of surface energy')
# plt.xlabel('# of layers')
# plt.ylabel('Surface energy [eV]')

# plt.show()

Conclusion: A vacuum with the $7$ Angstom is good enough.

### The convergence of frozen layers

In [ ]:
# # Build flow form inputs.

# # range(0,7,2)
# n_frozen_list = [0, 2, 4, 6]
# inputs = [relax_100_surface_input(n=8, vacuum=7.0, n_frozen=n) for n in n_frozen_list]

# flow = flowtk.Flow.from_inputs('07_relax_100_8_nfrozen', inputs=inputs, task_class=flowtk.RelaxTask)
# # flow.show_status()
# # flow.get_graphviz()

In [ ]:
# flow.rmtree()
# scheduler = flow.make_scheduler()
# scheduler.start()

# flow.build(abivalidate=True)
# flow.build_and_pickle_dump(abivalidate=True)

In [ ]:
# robot = abilab.GsrRobot.from_dir("07_relax_100_8_nfrozen/w0")

# energies = []
# surface_eneries = []

# for ind, calc in sorted(robot.items()):
#     energies.append(calc.energy_per_atom)
#     surface_eneries.append(surface_energy(calc.structure, calc.energy, total_energy))

In [ ]:
# energies

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_frozen_list, energies, 'o-')
# plt.title('Energy per atom')
# plt.xlabel('# of layers')
# plt.ylabel('eV')
# plt.show()

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_frozen_list, [energies[0]/e*100 for e in energies], 'o-')
# plt.title('energy difference in percentages')
# plt.xlabel('# of layers')
# plt.ylabel('%')
# plt.show()

In [ ]:
# surface_eneries

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_frozen_list, surface_eneries, 'o-')
# plt.title('Surface energy')
# plt.xlabel('# of layers')
# plt.ylabel('eV')
# plt.show()

In [ ]:
# # Build flow form inputs.
# # range(1,8,2)
# n_frozen_list = [0, 1, 3, 5, 7]
# inputs = [relax_100_surface_input(n=9, vacuum=7.0, n_frozen=n) for n in n_frozen_list]

# flow = flowtk.Flow.from_inputs('08_relax_100_9_nfrozen', inputs=inputs, task_class=flowtk.RelaxTask)
# # flow.show_status()
# # flow.get_graphviz()

In [ ]:
# flow.rmtree()
# scheduler = flow.make_scheduler()
# scheduler.start()

# flow.build(abivalidate=True)
# flow.build_and_pickle_dump(abivalidate=True)

In [ ]:
# robot = abilab.GsrRobot.from_dir("08_relax_100_9_nfrozen/w0")

# energies = []
# surface_eneries = []

# for ind, calc in sorted(robot.items()):
#     energies.append(calc.energy_per_atom)
#     surface_eneries.append(surface_energy(calc.structure, calc.energy, total_energy))

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_frozen_list, energies, 'o-')
# plt.title('Energy per atom')
# plt.xlabel('# of layers')
# plt.ylabel('eV')
# plt.show()

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_frozen_list, [energies[0]/e*100 for e in energies], 'o-')
# plt.title('energy difference in percentages')
# plt.xlabel('# of layers')
# plt.ylabel('%')
# plt.show()

In [ ]:
# plt.figure(figsize=(15,6))
# plt.plot(n_frozen_list, surface_eneries, 'o-')
# plt.title('Surface energy')
# plt.xlabel('# of layers')
# plt.ylabel('eV')
# plt.show()

## Macroscopic dielectric tensor and Born effective charges

Our calculations includes the response of the system to an external electric field.
The code below extracts the macroscopic dielectric tensor (`emacro`)
and the Born effective charges (`becs`) from the DDB file:

In [ ]:
# emacro, becs = ddb.anaget_epsinf_and_becs()

In [ ]:
# becs

As explained in the references, the Born effective charges must fulfill 
the charge neutrality sum-rule.
This rule is usually broken due to the discretization introduced by the FFT mesh, and `anaddb` will enforce it if `chneut` is set to 1 (default behaviour). Let's check it out!

In [ ]:
# emacro, becs_chneut0 = ddb.anaget_epsinf_and_becs(chneut=0)
# print(becs_chneut0)